# How to Scrape Data From Facebook Page Posts for Statistical Analysis

Following [Max Woolf (@minimaxir)](http://minimaxir.com/)

In [1]:
import urllib2
import json
import datetime
import csv
import time

In [2]:
app_id = "159957317871681"
app_secret = "2291402594fa74f6cf7daeff6c055f8e" # DO NOT SHARE WITH ANYONE!
# a little bit wierd and funny
access_token = app_id + "|" + app_secret

Let's scrap page https://www.facebook.com/nytimes

In [4]:
page_id = 'nytimes'

def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageData(page_id, access_token)

{
    "id": "5281959998", 
    "name": "The New York Times"
}


a getter that wait until succeed (no [HTTP Error 500 (Internal Error)](http://www.checkupdown.com/status/E500.html) error during connection)

In [5]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [6]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2017-05-11T17:21:06+0000", 
            "id": "5281959998_10151167417744999", 
            "message": "President Donald J. Trump called James Comey \"a showboat\" and \"a grandstander.\""
        }, 
        {
            "created_time": "2017-05-11T17:02:13+0000", 
            "id": "5281959998_10151167345799999", 
            "message": "Help us solve The New York Times Crossword using our web game. \u201cWordplay\u201d columnist Deb Amlen is here with hints. Put your guesses in the comments, and share this with your friends.", 
            "story": "The New York Times was live."
        }, 
        {
            "created_time": "2017-05-11T16:55:00+0000", 
            "id": "5281959998_10151167242304999", 
            "message": "He noticed the map in Walt Disney's office and asked if he could take it. His boss said sure. Now it might fetch between $750,000 and $1 million at auction."
        }, 
        {
            "created_t

In [9]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print json.dumps(test_status, indent=4, sort_keys=True)

{
    "comments": {
        "data": [
            {
                "created_time": "2017-05-11T17:21:56+0000", 
                "from": {
                    "id": "291559067963031", 
                    "name": "Brandon Horan"
                }, 
                "id": "10151167417744999_10151167421799999", 
                "message": "Anyone who defends this guy needs a mental evaluation...and sorry calling people liberals and snowflakes doesn't  excuse this mans actions."
            }
        ], 
        "paging": {
            "cursors": {
                "after": "NzM3", 
                "before": "NzM3"
            }, 
            "next": "https://graph.facebook.com/v2.9/5281959998_10151167417744999/comments?access_token=159957317871681%7C2291402594fa74f6cf7daeff6c055f8e&summary=true&limit=1&after=NzM3"
        }, 
        "summary": {
            "can_comment": false, 
            "order": "ranked", 
            "total_count": 737
        }
    }, 
    "created_time": "2017-05-

creating vectors like (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

In [10]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status

(u'5281959998_10151167417744999', 'President Donald J. Trump called James Comey "a showboat" and "a grandstander."', 'Trump Calls Comey a \xe2\x80\x9cShowboat,\xe2\x80\x9d Says he Decided on His Own to Fire Him', u'link', u'https://www.nytimes.com/2017/05/11/us/politics/trump-comey-showboat-fbi.html?smid=fb-nytimes&smtyp=cur', '2017-05-11 12:21:06', 250, 737, 288)


Now we just need to:

1. Query each page of Facebook Page Statuses (100 statuses per page) using `getFacebookPageFeedData`.
2. Process all statuses on that page using `processFacebookPageFeedStatus` and writing the output to a CSV file.
3. Navigate to the next page, and repeat until no more statuses



In [11]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping nytimes Facebook Page: 2017-05-11 20:46:48.691307



KeyboardInterrupt: 